In [1]:
import os
import sys
PROJECT_ROOT = os.path.abspath('/workspaces/juyeon-docker/SVD_FMs-main')
sys.path.append(PROJECT_ROOT)

In [2]:
import pickle
import random

import numpy as np
import pytorch_lightning as pl
import torch
import torch.nn as nn

import src

In [10]:
def setseed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)    
    torch.cuda.manual_seed_all(seed)
setseed(seed=42)

In [11]:
with open('../d_pickle/args_original_fm.pickle', mode='rb') as fr:
    args_original_fm = pickle.load(fr)
with open('../d_pickle/dataset_original_fm.pickle', mode='rb') as fr:
    dataset_original_fm = pickle.load(fr)
with open('../d_pickle/dataloader_original_fm.pickle', mode='rb') as fr:
    dataloader_original_fm = pickle.load(fr)
with open('../d_pickle/model_original_fm.pickle', mode='rb') as fr:
    model_original_fm = pickle.load(fr)

In [5]:
trainer = pl.Trainer(max_epochs=args_original_fm.num_epochs_training, enable_checkpointing=False, logger=True)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [6]:
device = torch.device("cuda") # torch.cuda.is_available()
model_original_fm.to(device)

FM(
  (embedding): FeatureEmbedding(
    (embedding): Embedding(2688, 16)
  )
  (linear): FM_Linear(
    (linear): Embedding(2665, 1)
  )
  (interaction): FM_Interaction()
  (bceloss): BCEWithLogitsLoss()
  (sig): Sigmoid()
  (last_linear): Linear(in_features=2, out_features=1, bias=True)
)

In [12]:
# confiure_optimizer를 직접 호출
optimizer = model_original_fm.configure_optimizers()

In [13]:
for epoch in range(1):
    model_original_fm.train() # type(model_original_fm) == src.model.original.fm.FM
    train_loss = 0.0
    for batch_idx, batch in enumerate(dataloader_original_fm): # type(batch)==list
        batch = [item.to(device) for item in batch]
        break
        # break
        # optimizer.zero_grad()
        # output = model_original_fm.training_step(batch, batch_idx)
        # loss = output
        # loss.backward()
        # optimizer.step()

        # train_loss += loss.item()
    print(f'[Epoch {epoch+1}] Training Loss: {train_loss / len(dataloader_original_fm)}')

[Epoch 1] Training Loss: 0.0


In [14]:
class FeatureEmbedding(nn.Module):
    def __init__(self, args, field_dims):
        super(FeatureEmbedding, self).__init__()
        self.embedding = nn.Embedding(sum(field_dims+1), args.emb_dim, device='cuda') # (2688, 16)
        self.field_dims = field_dims
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.int64) # field_dims = [1, 6, 3, 2] -> offsets = [0, 1, 7, 10]

    def forward(self, x):
        # 'new_tensor'는 기존 텐서의 dtype(데이터 타입)과 device를 그대로 물려받는 새로운 텐서를 생성 
        x = x + x.new_tensor(self.offsets).unsqueeze(0) 
        x = self.embedding(x)
        return x
embedding = FeatureEmbedding(args_original_fm, field_dims)

NameError: name 'field_dims' is not defined

In [103]:
class FM_Linear(nn.Module):

    def __init__(self, args, field_dims):
        super(FM_Linear, self).__init__()
        self.linear = torch.nn.Embedding(sum(field_dims)+1, 1, device='cuda')
        self.bias = nn.Parameter(torch.randn(1))
        self.w = nn.Parameter(torch.randn(args.cont_dims))
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.int64)
        self.args = args
    
    def forward(self, x, x_cont):
        x = x + x.new_tensor(self.offsets).unsqueeze(0)
        self.tmp_x_1 = x
        linear_term = self.linear(x)
        self.linear_term = linear_term
        cont_linear = torch.matmul(x_cont, self.w).reshape(-1, 1) # add continuous features
        
        x = torch.sum(linear_term, dim=1) + self.bias
        self.tmp_x_2 = x
        x = x + cont_linear 
        self.tmp_x_3 = x
        return x

linear = FM_Linear(args_original_fm, field_dims)

In [99]:
lin_term = linear(x, x_cont)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat in method wrapper_CUDA_addmv_)

In [ ]:
def forward(self, x, x_cont, emb_x):
    # FM part loss with interaction terms
    # x: batch_size * num_features
    lin_term = self.linear(x=x, x_cont=x_cont)
    inter_term, cont_emb = self.interaction(emb_x, x_cont)
    lin_term_sig = self.sig(lin_term)
    inter_term_sig = self.sig(inter_term)
    outs = torch.cat((lin_term_sig, inter_term_sig), 1)
    x = self.last_linear(outs)
    x = x.squeeze(1)
        
    return x, cont_emb, lin_term, inter_term

In [67]:
x, x_cont, y, c_values = batch
embed_x = embedding(x)

In [68]:
embed_x

tensor([[[ 0.7225, -0.3073,  0.3096,  ..., -1.5180,  0.1782,  0.2136],
         [-0.7349, -0.4854,  0.8096,  ...,  1.0822, -1.7898,  0.8658],
         [ 0.7755, -1.3483, -0.7340,  ..., -1.4865,  1.8129,  0.1935],
         ...,
         [-1.2306, -0.8370, -0.2251,  ..., -1.4198, -1.0235,  0.0647],
         [ 0.5580,  1.6420,  0.3771,  ..., -1.6685,  1.0970,  1.2572],
         [ 0.4353, -0.1387, -0.1793,  ..., -0.7133, -0.8459, -0.1945]],

        [[-1.0001, -0.8304,  0.1950,  ..., -0.4996,  0.6328, -0.3062],
         [-0.8531,  0.4859, -0.5336,  ...,  2.1535,  1.1598, -0.6850],
         [ 0.7755, -1.3483, -0.7340,  ..., -1.4865,  1.8129,  0.1935],
         ...,
         [-0.9086, -1.8216, -0.8330,  ..., -0.8399,  0.8780,  0.5973],
         [ 0.5580,  1.6420,  0.3771,  ..., -1.6685,  1.0970,  1.2572],
         [-0.5074,  0.7375, -0.3458,  ..., -2.2177,  0.5445,  0.1553]],

        [[ 1.0085,  0.4958,  0.1189,  ..., -0.3998,  1.5469,  1.9037],
         [-0.1219, -0.9968, -1.6285,  ..., -1

In [17]:
print(x.shape)
print('x[0]\n', x.cpu().numpy()[0])
print(x.new_tensor(embedding.offsets).unsqueeze(0).shape)
x = x + x.new_tensor(embedding.offsets).unsqueeze(0)
print('x + x.new_tensor(offset)\n', x.cpu().numpy()[0])

torch.Size([4096, 24])
x[0]
 [ 894 1248    0    0    0    0    0    0    0    0    1    0    0    0
    0    0    0    0    0    0    0    4    0   10]
torch.Size([1, 24])
x + x.new_tensor(offset)
 [ 894 2191 2593 2595 2597 2599 2601 2603 2605 2607 2610 2611 2613 2615
 2617 2619 2621 2623 2625 2627 2629 2635 2641 2653]


In [ ]:
# print('filed_dims\n', embedding.field_dims, len(embedding.field_dims))

print('x[0]\n', x.cpu().numpy()[0])
print('offsets\n', embedding.offsets, len(embedding.offsets))
print('x + x.new_tensor(offset)\n', tmp_x.cpu().numpy()[0])

In [ ]:
# def training_step(self, batch, batch_idx):
#     x, x_cont, y, c_values = batch
#     embed_x = self.embedding(x)
#     y_pred, _, _, _ = self.forward(x, x_cont, embed_x)
#     loss_y = self.loss(y_pred, y, c_values)
#     self.log('train_loss', loss_y, on_step=True, on_epoch=True, prog_bar=True, logger=True)
#     return loss_y